In [ ]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [47]:
#####Import Relevant Packages

#Asana
import asana
import csv
import sys
import pandas as pd

#Sheets
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [48]:
#####Configure Asana and Export Tasks
outputdir = ''
outputfilename = 'asana-export.csv'

personal_access_token = '0/127b1b5171a9372ec4e53dd9e6879c34'
client = asana.Client.access_token(personal_access_token) #start session
client.headers={'asana-enable': 'string_ids,new_sections'}
me = client.users.me() #get user info
workspace = me['workspaces'][0] #set main workspace
print('Initialising Asana session for ' + me['name'] + ' in workspace: ' + workspace['name'])
projects = client.projects.find_by_workspace(workspace['gid'], iterator_type=None) #find projects within workspace
tasklist = []

print('\nLooping through all the tasks within the projects...\n')
for project in projects:
    print(project['name'])
    #opt_expand delivers all attributes (not a compact view) for each field    
    tasks = client.tasks.find_by_project(project['gid'], {"opt_expand":"name, \
        projects, workspace, gid, due_on, created_at, modified_at, completed, \
        completed_at, assignee, parent, notes, tags"}, iterator_type=None)
    
    for task in tasks:
        if task['completed'] == False: #build list of only open tasks
            #Truncate notes if needed
            #if len(task['notes']) > 80:
                #task['notes'] = task['notes'][:79]

            #Loop through tags applied to each task, extract from json and turn into comma separated string
            tags = task['tags']         
            if task['tags'] is not None:
                tagname=''
                i=0
                for tag in tags:
                    if i==0:
                        tagname = tag['name']
                    else:
                        tagname = tagname + ', ' + tag['name']
                    i=i+1        
            
            #deal with assignee being blank
            assignee = task['assignee']['name'] if task['assignee'] is not None else ''

            #cleaning up the dates so they're readable
            created_at = task['created_at'][0:10] + ' ' + task['created_at'][11:16] if \
                    task['created_at'] is not None else None
            modified_at = task['modified_at'][0:10] + ' ' + task['modified_at'][11:16] if \
                    task['modified_at'] is not None else None
            completed_at = task['completed_at'][0:10] + ' ' + task['completed_at'][11:16] if \
                task['completed_at'] is not None else None
            #build row
            row = [task['name'], project['name'], task['due_on'], created_at, \
                modified_at, task['completed'], completed_at, assignee, \
                task['parent'], task['notes'], task['gid'], tagname]
            row = ['' if s is None else s for s in row]
            tasklist.append(row)
            
AsanaData = {}
AsanaData['values']= tasklist
AsanaData['majorDimension']="ROWS"
AsanaData['range']= 'TestSheet!A2:L10'

Initialising Asana session for Kevin Clyne in workspace: hraadvisors.com

Looping through all the tasks within the projects...

WeWork GIR 2020


In [49]:
######Configure Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of test spreadsheet.
SAMPLE_SPREADSHEET_ID = '12rGg_6-N_e33zVElZJt75HToATJDYkkTjVGssXyh7ik'
SAMPLE_RANGE_NAME = 'TestSheet!A2:L10'
value_input_option='USER_ENTERED'

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()

service.spreadsheets().values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                                range=SAMPLE_RANGE_NAME, valueInputOption=value_input_option, body=AsanaData).execute()


{'range': 'TestSheet!A2:L10', 'majorDimension': 'ROWS'}


In [ ]:
#####For Final Python Executable, Do not compile

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

SAMPLE_SPREADSHEET_ID = '12rGg_6-N_e33zVElZJt75HToATJDYkkTjVGssXyh7ik'
SAMPLE_RANGE_NAME = 'TestSheet!A2:L'

def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()

    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        print('Name, Major:')
        for row in values:
            print('%s, %s' % (row[0], row[4]))

if __name__ == '__main__':
    main()

In [ ]:
###Export to CSV Scrap

print('\nExporting to csv file: ' + outputdir + outputfilename + '...') 
csvheader = ['Task', 'Projects', 'DueDate', 'CreatedAt', \
    'ModifiedAt', 'Completed', 'CompletedAt', 'Assignee', \
    'Parent', 'Notes', 'TaskId', 'Tags']
with open(outputdir + outputfilename, 'w', encoding='utf8') as csvfile:
    csvwriter = csv.writer(csvfile, lineterminator='\n', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(csvheader)
    for item in tasklist:
        csvwriter.writerow(item)

print('\nFinished!')